In [61]:
!pip install folium
import pandas as pd
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

### Cargamos y preparamos los datos

In [124]:
df = pd.read_csv('delitos.csv')

#Borramos las columnas sin datos
df.drop(['lugar'], axis = 1, inplace=True)
df.drop(['origen_dato'], axis = 1, inplace=True)

#Completamos los datos nulos
df['hora'].fillna('99:99:99', inplace = True)
df['comuna'].fillna('N/A', inplace = True)
df['barrio'].fillna('N/A', inplace = True)

#Damos formato a la fecha
df['fecha_for'] = pd.to_datetime(df['fecha'])
#Extraemos dia de la semana
df['dia_semana'] = df['fecha_for'].dt.dayofweek

#Creamos  un dataframe indexado por fecha
df.index = pd.DatetimeIndex(df['fecha_for'])

#creamos las columnas de anho, mes y dia
df['anho'] = df['fecha_for'].dt.year 
df['mes'] = df['fecha_for'].dt.month 
df['dia'] = df['fecha_for'].dt.day
df['solo_hora'] = df['hora'].apply(lambda s: int(s.split(':')[0]))

len(df)


350980

## Mapa de Calor del 01/09/2018

In [125]:
# Seleccionamos solo las columnas de coordenadas validas para la fecha seleccionada
df = df[(df['latitud'] != 0) & (df['longitud'] != 0) & (df['fecha_for'] == '2018-09-01')  ]

#Creamos un campo auxiliar
df['count'] = 1

# Creamos un mapa en el centro de CABA
map_van = folium.Map(location= [-34.60, -58.42], zoom_start = 12)



# Otro metodo
#heat_data = [[row['latitud'],row['longitud'], row['count']] for index, row in df.iterrows()]
#HeatMap(heat_data).add_to(map_van)
#map_van

HeatMap(data=df[['latitud', 'longitud', 'count']].groupby(['latitud', 'longitud']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(map_van)
map_van

## Mapa de Calor por Hora del Dia de los Meses (Ene a Mar/2018)

In [ ]:
# Seleccionamos solo las columnas de coordenadas validas para la fecha seleccionada
df = df[(df['latitud'] != 0) & (df['longitud'] != 0) & (df['anho'] == 2018) & (df['mes'] <= 3) & (df['solo_hora'] != 99)]

In [121]:
df_hour_list = []
for hour in df.solo_hora.sort_values().unique():
    df_hour_list.append(df.loc[df.solo_hora == hour, ['latitud', 'longitud', 'count']].groupby(['latitud', 'longitud']).sum().reset_index().values.tolist())

In [122]:
base_map = folium.Map(location= [-34.60, -58.42], zoom_start = 12)
HeatMapWithTime(df_hour_list, radius=9, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.1, max_opacity=0.8, use_local_extrema=True).add_to(base_map)
base_map